In [1]:
!pip install catboost

     |████████████████████████████████| 67.3MB 54kB/s 


In [2]:
!pip install nltk

In [3]:
!pip install mlxtend

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('punkt')
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_score, recall_score, precision_recall_curve
from matplotlib import pyplot as plt
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import roc_auc_score
import numpy as np
from sklearn.model_selection import GridSearchCV

from catboost import CatBoostClassifier
from mlxtend.classifier import EnsembleVoteClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


In [5]:
with open("dataset_ok.txt", "r+") as file:
  file_readen = file.read().split("\n")

In [105]:
labels = list(map(lambda x: list(map(lambda y: y[9 :], x.split(" ")[0].split(","))), file_readen))[: -1]
text = list(map(lambda x: " ".join(x.split(" ")[1 :]), file_readen))[: -1]

In [106]:
df = pd.DataFrame(np.array([labels, text]).T, columns = ["label", "text"])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [159]:
def onehot(series, unique):
  series_new = pd.DataFrame(data = np.array([[0 for j in range(len(series))] for i in range(len(unique))]).T ,columns = unique)
  for elIndex in range(len(series)):
      for el_class in series.iloc[elIndex]: series_new["{}".format(el_class)].iloc[elIndex] = 1
  return series_new

In [160]:
df_onehoted = onehot(df.label, ['NORMAL', 'INSULT', 'THREAT', 'OBSCENITY'])

In [163]:
df_onehoted.INSULT.value_counts()

0    211464
1     36826
Name: INSULT, dtype: int64

In [164]:
df_onehoted.THREAT.value_counts()

0    236263
1     12027
Name: THREAT, dtype: int64

In [168]:
df_onehoted.OBSCENITY.value_counts()

0    244029
1      4261
Name: OBSCENITY, dtype: int64

In [170]:
df_onehoted

,NORMAL,INSULT,THREAT,OBSCENITY
0,0,1,0,0
1,1,0,0,0
2,1,0,0,0
3,1,0,0,0
4,1,0,0,0
...,...,...,...,...
248285,1,0,0,0
248286,0,1,0,0
248287,1,0,0,0
248288,1,0,0,0


In [34]:
df.iloc[248286].text

'ёбанные нубы заходите на сервер мой ник _creepro для пвп всех переебу если вы пролистываете значит вы зассали лохи все подробности в сообщениях'

In [ ]:
# df_2ch = pd.read_csv("labeled.csv")
# df_2ch['toxic'] = df_2ch['toxic'].astype(int)
# df["toxic"] = df["toxic"].apply(int)

In [166]:
snowball = SnowballStemmer(language="russian")
nltk.download('stopwords')
russian_stop_words = stopwords.words("russian")

def tokenize_sentence(sentence: str, remove_stop_words: bool = True):
    tokens = word_tokenize(sentence, language="russian")
    tokens = [i for i in tokens if i not in string.punctuation]
    if remove_stop_words:
        tokens = [i for i in tokens if i not in russian_stop_words]
    tokens = [snowball.stem(i) for i in tokens]
    return tokens

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [172]:
X_train_ins, X_test_ins, y_train_ins, y_test_ins = train_test_split(df.text, df_onehoted.INSULT, test_size=0.33, random_state=42)
X_train_thr, X_test_thr, y_train_thr, y_test_thr = train_test_split(df.text, df_onehoted.THREAT, test_size=0.33, random_state=42)
X_train_obs, X_test_obs, y_train_obs, y_test_obs = train_test_split(df.text, df_onehoted.OBSCENITY, test_size=0.33, random_state=42)

In [174]:
models_pipeline = {
    "INS": Pipeline([("vectorizer", TfidfVectorizer(tokenizer=tokenize_sentence)), ("model", LogisticRegression(random_state=42, C=10))]),
    "THR": Pipeline([("vectorizer", TfidfVectorizer(tokenizer=tokenize_sentence)), ("model", LogisticRegression(random_state=42, C=10))]), 
    "OBS": Pipeline([("vectorizer", TfidfVectorizer(tokenizer=tokenize_sentence)), ("model", LogisticRegression(random_state=42, C=10))])
    }

In [175]:
models_pipeline['INS'].fit(X_train_ins, y_train_ins)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_patt...w+\\b',
                                 tokenizer=<function tokenize_sentence at 0x7fe945916f80>,
                                 use_idf=True, vocabulary=None)),
                ('model',
                 LogisticRegression(C=10, class_weight=None, dual=False,
                                    fit

In [186]:
models_pipeline['THR'].fit(X_train_thr, y_train_thr)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_patt...w+\\b',
                                 tokenizer=<function tokenize_sentence at 0x7fe945916f80>,
                                 use_idf=True, vocabulary=None)),
                ('model',
                 LogisticRegression(C=10, class_weight=None, dual=False,
                                    fit

In [187]:
models_pipeline['OBS'].fit(X_train_obs, y_train_obs)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_patt...w+\\b',
                                 tokenizer=<function tokenize_sentence at 0x7fe945916f80>,
                                 use_idf=True, vocabulary=None)),
                ('model',
                 LogisticRegression(C=10, class_weight=None, dual=False,
                                    fit

In [189]:
print("Precision: {}; Recall: {}".format(precision_score(y_true=y_test_ins, y_pred=models_pipeline['INS'].predict(X_test_ins), average = 'weighted'), recall_score(y_true=y_test_ins, y_pred=models_pipeline['INS'].predict(X_test_ins), average = 'weighted')))

Precision: 0.9599445125554553; Recall: 0.960676625659051


In [190]:
print("Precision: {}; Recall: {}".format(precision_score(y_true=y_test_thr, y_pred=models_pipeline['THR'].predict(X_test_thr), average = 'weighted'), recall_score(y_true=y_test_thr, y_pred=models_pipeline['THR'].predict(X_test_thr), average = 'weighted')))

Precision: 0.9774182679964719; Recall: 0.9786784807654755


In [191]:
print("Precision: {}; Recall: {}".format(precision_score(y_true=y_test_obs, y_pred=models_pipeline['OBS'].predict(X_test_obs), average = 'weighted'), recall_score(y_true=y_test_obs, y_pred=models_pipeline['OBS'].predict(X_test_obs), average = 'weighted')))

Precision: 0.9913580079452491; Recall: 0.9919693419254052


In [ ]:
# model_pipeline = Pipeline([
#     ("vectorizer", TfidfVectorizer(tokenizer=tokenize_sentence)),
#     ("model", LogisticRegression(random_state=42, C=10)), # C = 10
#     # ("model", EnsembleVoteClassifier(clfs = [LogisticRegression(random_state=42), RandomForestClassifier(random_state = 42), SVC(probability = True)], weights = [2, 1, 1], voting = 'soft'))
#     # ("model", SVC(random_state = 42, probability = True))
#     # ("model", RandomForestClassifier(random_state = 42))
#     # ("model", CatBoostClassifier(random_state = 42))
# ])

In [176]:
# model_pipeline.fit(train_df["comment"], train_df["toxic"])
# model_pipeline.fit(X_train, y_train)

In [ ]:
model_pipeline[0].predict(["всем привет"])

array(['NORMAL'], dtype=object)

In [ ]:
print("Precision: {}; Recall: {}".format(precision_score(y_true=y_test, y_pred=model_pipeline.predict(X_test), average = 'weighted'), recall_score(y_true=y_test, y_pred=model_pipeline.predict(X_test), average = 'weighted')))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision: 0.9530974626605782; Recall: 0.9549165822522182


In [ ]:
# prec, rec, thresholds = precision_recall_curve(y_test, model_pipeline.predict_proba(X_test).T[1])

In [ ]:
# plot_precision_recall_curve(estimator=model_pipeline, X=test_df["comment"], y=test_df["toxic"])

In [ ]:
# np.where(prec > 0.95)

In [188]:
from joblib import dump, load

dump(models_pipeline["INS"], 'model_ins_logistic_2.joblib')
dump(models_pipeline["THR"], 'model_thr_logistic_2.joblib')
dump(models_pipeline["OBS"], 'model_obs_logistic_2.joblib')

['model_obs_logistic_2.joblib']

In [ ]:
# model = load('model_logistic_1.joblib')

array(['NORMAL'], dtype=object)